### Miscellaneous


In [1]:
%run core.ipynb

#### Code Snippets

In [33]:
import matplotlib
import matplotlib.pyplot as plot

print(matplotlib.style.available)
matplotlib.style.use("seaborn-deep")
matplotlib.style.use("default")

['Solarize_Light2', '_classic_test_patch', 'bmh', 'classic', 'dark_background', 'fast', 'fivethirtyeight', 'ggplot', 'grayscale', 'seaborn', 'seaborn-bright', 'seaborn-colorblind', 'seaborn-dark', 'seaborn-dark-palette', 'seaborn-darkgrid', 'seaborn-deep', 'seaborn-muted', 'seaborn-notebook', 'seaborn-paper', 'seaborn-pastel', 'seaborn-poster', 'seaborn-talk', 'seaborn-ticks', 'seaborn-white', 'seaborn-whitegrid', 'tableau-colorblind10']


#### Triangle Count

In [34]:
def triangles(n):
    if n == 0:
        return 1
    s, k = 0, n
    while k > 0:
        s += k
        k -= 2
    return triangles(n - 1) + s + int((n + 2) * (n + 1) / 2)

In [35]:
for n in range(25):
    print(f"n = {n}:	{triangles(n)}")

n = 0:	1
n = 1:	5
n = 2:	13
n = 3:	27
n = 4:	48
n = 5:	78
n = 6:	118
n = 7:	170
n = 8:	235
n = 9:	315
n = 10:	411
n = 11:	525
n = 12:	658
n = 13:	812
n = 14:	988
n = 15:	1188
n = 16:	1413
n = 17:	1665
n = 18:	1945
n = 19:	2255
n = 20:	2596
n = 21:	2970
n = 22:	3378
n = 23:	3822
n = 24:	4303


#### Turing Machines

In [34]:
from array import array


class Tape:

    def __init__(self, values):
        self.values = array("I", values)
        self.pos = 0
        
    def __repr__(self):
        return " ".join([f"{v}" for v in self.values[:self.pos]] + 
                        [_blu(self.values[self.pos])] + 
                        [f"{v}" for v in self.values[self.pos + 1:]])
    
    def __getitem__(self, i):
        return self.values[i]
    
    def read(self):
        return self.values[self.pos]

    def write(self, v):
        self.values[self.pos] = v

    def move(self, mv):
        if mv == "L":
            if self.pos == 0:
                self.values.insert(0, 0)
            else:
                self.pos -= 1
        else:
            if self.pos == len(self.values) - 1:
                self.values.append(0)
            self.pos += 1


In [35]:
class TuringMachine:
    
    def __init__(self, instructions):
        self.instructions = instructions
        
    def __getitem__(self, i):
        return self.instructions[i]

    def __call__(self, tape, max_iterations=100):
        i, k, mv = 0, 0, None
        while not mv == "S":
            if i > max_iterations:
                raise StopIteration(f"max iterations {max_iterations} reached")
            value = tape.read()
            k, new_value, mv = self.instructions[k][value]
            tape.write(new_value)
            tape.move(mv)
            i += 1
            yield k, value, new_value, mv

In [36]:
def run_turing_machine(tm, tape):
    print(f"start: {tape}")    
    print()
    for i, r, w, move in tm(tape):
        print(f"{i}: {r}->{w} mv={move}:", tape)
    print()
    print(f"result: {tape}")

In [37]:
UN_PLUS_ONE = {
    0: {
        0: (0, 0, "R"), 
        1: (1, 1, "R"),
    },
    1: {
        0: (0, 1, "S"), 
        1: (1, 1, "R"),
    },
}
un_plus_one = TuringMachine(UN_PLUS_ONE)

In [38]:
tape = Tape([0, 0, 1, 1, 1, 0, 0])
run_turing_machine(un_plus_one, tape)

start: 0 0 1 1 1 0 0

0: 0->0 mv=R: 0 0 1 1 1 0 0
0: 0->0 mv=R: 0 0 1 1 1 0 0
1: 1->1 mv=R: 0 0 1 1 1 0 0
1: 1->1 mv=R: 0 0 1 1 1 0 0
1: 1->1 mv=R: 0 0 1 1 1 0 0
0: 0->1 mv=S: 0 0 1 1 1 1 0

result: 0 0 1 1 1 1 0


In [39]:
tape = Tape([0, 1, 1, 1, 1, 1, 0, 0])
run_turing_machine(un_plus_one, tape)

start: 0 1 1 1 1 1 0 0

0: 0->0 mv=R: 0 1 1 1 1 1 0 0
1: 1->1 mv=R: 0 1 1 1 1 1 0 0
1: 1->1 mv=R: 0 1 1 1 1 1 0 0
1: 1->1 mv=R: 0 1 1 1 1 1 0 0
1: 1->1 mv=R: 0 1 1 1 1 1 0 0
1: 1->1 mv=R: 0 1 1 1 1 1 0 0
0: 0->1 mv=S: 0 1 1 1 1 1 1 0

result: 0 1 1 1 1 1 1 0


In [40]:
UN_TIMES_TWO = {
    0: {
        0: (0, 0, "R"), 
        1: (1, 0, "R"),
    },
    1: {
        0: (2, 1, "L"), 
        1: (1, 1, "R"),
    },
    2: {
        0: (3, 0, "R"), 
        1: (4, 0, "R"),
    },
    3: {
        0: (0, 1, "S"), 
        1: (3, 1, "R"),
    },
    4: {
        0: (5, 1, "L"), 
        1: (4, 1, "R"),
    },
    5: {
        0: (2, 1, "L"), 
        1: (5, 1, "L"),
    },
}
un_times_two = TuringMachine(UN_TIMES_TWO)

In [41]:
tape = Tape([0, 1, 1, 1, 0])
tape = Tape([1, 1, 1])
run_turing_machine(un_times_two, tape)

start: 1 1 1

1: 1->0 mv=R: 0 1 1
1: 1->1 mv=R: 0 1 1
1: 1->1 mv=R: 0 1 1 0
2: 0->1 mv=L: 0 1 1 1
4: 1->0 mv=R: 0 1 0 1
4: 1->1 mv=R: 0 1 0 1 0
5: 0->1 mv=L: 0 1 0 1 1
5: 1->1 mv=L: 0 1 0 1 1
2: 0->1 mv=L: 0 1 1 1 1
4: 1->0 mv=R: 0 0 1 1 1
4: 1->1 mv=R: 0 0 1 1 1
4: 1->1 mv=R: 0 0 1 1 1
4: 1->1 mv=R: 0 0 1 1 1 0
5: 0->1 mv=L: 0 0 1 1 1 1
5: 1->1 mv=L: 0 0 1 1 1 1
5: 1->1 mv=L: 0 0 1 1 1 1
5: 1->1 mv=L: 0 0 1 1 1 1
2: 0->1 mv=L: 0 1 1 1 1 1
3: 0->0 mv=R: 0 1 1 1 1 1
3: 1->1 mv=R: 0 1 1 1 1 1
3: 1->1 mv=R: 0 1 1 1 1 1
3: 1->1 mv=R: 0 1 1 1 1 1
3: 1->1 mv=R: 0 1 1 1 1 1
3: 1->1 mv=R: 0 1 1 1 1 1 0
0: 0->1 mv=S: 0 1 1 1 1 1 1 0

result: 0 1 1 1 1 1 1 0
